In [1]:
#!pip install natsort
#!pip install matplotlib
#!pip install imageio

In [2]:
import os
import natsort
import imageio
from matplotlib import pylab as plt
import matplotlib.patches as patches

path = "/prm-mnt/osni/raw_data/Images/"

In [3]:
!ls /prm-mnt/osni/raw_data/Images/

 Drainage  'No Drainage'   Unknown


In [4]:
# Parse the directory to get Drainage + "NO Drainage" files

jpeg_files_path = [];
xml_files_path = [];

for subdir, dirs, files in os.walk(path):
    for file in files:
        if(".jpg" in file):
            if("Unknown" not in subdir):
                jpeg_files_path.append(os.path.join(subdir, file))
            
for subdir, dirs, files in os.walk(path):
    for file in files:
        if(".xml" in file):
            xml_files_path.append(os.path.join(subdir, file))
            
jpeg_files_path = natsort.natsorted(jpeg_files_path)
xml_files_path = natsort.natsorted(xml_files_path)

In [5]:
# Filter files with bounding boxes only

pathsPair = [];
for i in range(0, len(jpeg_files_path)):
    pathTmp = "";
    for j in range(0, len(xml_files_path)):
        if(jpeg_files_path[i][:-4] in xml_files_path[j]):
            pathTmp = xml_files_path[j];
    pathsPair.append([jpeg_files_path[i], pathTmp]);

In [6]:
# Number of jpg files and xml files
print (len(jpeg_files_path), len(xml_files_path))

969 467


In [7]:
# One path for a jpg file
jpeg_files_path[0]

'/prm-mnt/osni/raw_data/Images/Drainage/Track_B-Sphere-74.jpg'

In [8]:
# Paired list for "Drainage" only
pathsPair[0]

['/prm-mnt/osni/raw_data/Images/Drainage/Track_B-Sphere-74.jpg',
 '/prm-mnt/osni/raw_data/Images/Drainage/Track_B-Sphere-74.xml']

In [9]:
# Source: https://stackoverflow.com/a/53832130 by pix_1
# Source 2: https://cv.gluon.ai/_modules/gluoncv/data/pascal_voc/detection.html#VOCDetection

import xml.etree.ElementTree as ET

def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes

In [10]:
# Example output
name, boxes = read_content(pathsPair[0][1])
print (pathsPair[0])

['/prm-mnt/osni/raw_data/Images/Drainage/Track_B-Sphere-74.jpg', '/prm-mnt/osni/raw_data/Images/Drainage/Track_B-Sphere-74.xml']


In [13]:
# Name from the XML
name

'Track_B-Sphere-74.jpg'

In [14]:
# Bounding box annotations
boxes

[[2980, 2071, 3091, 2157], [4400, 2205, 4539, 2312]]

In [16]:
# Pairs with annotation only
pairsWithAnnotationOnly = [];
for i in range(0, len(pathsPair)):
    if(pathsPair[i][1] != ""):
        pairsWithAnnotationOnly.append(pathsPair[i])
        
print (len(pairsWithAnnotationOnly))

467


In [ ]:
# Visualize five images with as example

for i in [5,20,40,90,215]:
    fig, ax = plt.subplots(figsize=(40,40))
    
    name, boxes = read_content(pairsWithAnnotationOnly[i][1])

    for box in boxes:
        # Create a Rectangle patch
        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='r', facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.title(pairsWithAnnotationOnly[i][0])
    plt.imshow(imageio.imread(pairsWithAnnotationOnly[i][0]))
    plt.show()